In [0]:
create table books
(book_id string, book_name string, author string, category string, cost double)
using csv
options(
  delimiter = ';',
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv-new/',
  header = 'true'
);

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore

In [0]:
drop table books;

In [0]:
select * from books;

In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark-streaming") \
.getOrCreate()

In [0]:
%python
(spark.readStream.table("books").createOrReplaceTempView("streaming_books_dat"))

In [0]:
select * from streaming_books_dat;

In [0]:
create or replace Temp view streaming_authors_view as 
select author, count(book_id) as book_count
from streaming_books_dat
group by author;

In [0]:
select * from streaming_authors_view;

In [0]:
show tables;

In [0]:
%python
(spark.table("streaming_authors_view")
        .writeStream
        .trigger(processingTime="4 seconds")
        .outputMode("complete")
        .option("checkpointLocation","dbfs:/mnt/demo/streaming_authors_loc")
        .table("authors"))

In [0]:
select * from authors;

In [0]:
INSERT INTO books
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
         ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
         ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35);

In [0]:
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35);

In [0]:
%python
(spark.table("streaming_authors_view")
        .writeStream
        .trigger(availableNow=True)
        .outputMode("complete")
        .option("checkpointLocation","dbfs:/mnt/demo/streaming_authors_loc")
        .table("authors")
        .awaitTermination()
);

In [0]:
select * from authors;